In [8]:
import numpy as np
import os
dtype = np.float32
import torch
import torchvision
# import tqdm
import os

from transformers import CLIPVisionModelWithProjection, ViTForImageClassification, AutoModelForCausalLM
from transformers import AutoModel, AutoTokenizer
import numpy

from huggingface_hub import try_to_load_from_cache, _CACHED_NO_EXIST
from huggingface_hub import scan_cache_dir

### 2. 모델의 tensor를 모아서 자르고 TFRecord로 저장
tensor 별로 train, val split

In [9]:

# filters = ['mlp', 'attn']
filters = ['layers-0', 'q_proj']
model_names = ['meta-llama/Meta-Llama-3-8B']
tensor_zoo_path = '/home/jgryu/Weight_compression/Wparam_dataset/model_param_tensor/'
save_path = '/home/jgryu/Weight_compression/Wparam_dataset/TFRecord/'

# # model_name = model_names[0]
# model_name = model_names[0]

In [10]:
import glob
import random
import json
import tensorflow as tf
from tqdm import tqdm

def check_contains_all_substrings(string, substrings):
    return all(substring in string for substring in substrings)

def check_contains_any_substrings(string, substrings):
    return any(substring in string for substring in substrings)

def serialize_example(slice):
    feature_dict = {
        'slice': tf.train.Feature(float_list=tf.train.FloatList(value=slice))
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature_dict))
    return example_proto.SerializeToString()

chunk_direction = 'out' ## 같은 Out channel 요소들이 같이 묶이게 차름
# chunk_direction = 'in'

for model_name in model_names:
    print(model_name)
    for d in [4096, 1024, 256, 128, 64, 32, 16]:
        print(f'dim = {d}')
        target_model_tensor_path = tensor_zoo_path + model_name
        tensor_path_list = glob.glob(f'{target_model_tensor_path}/**/*.npy', recursive=True)

        random.seed(100)

        # all_tensors = []
        tensors_train = []
        tensors_val = []

        for tensor_path in tqdm(tensor_path_list):
            if not check_contains_all_substrings(tensor_path, filters):
                continue
            print(tensor_path.split('/')[-1])
            t = np.load(tensor_path)
            # print(t.shape)
            if chunk_direction == 'in':
                t = t.T
            t = t.astype(np.float32)
            t = t.reshape(-1, d)
            # print(t.shape)
            # all_tensors.append(t)
            
            indices = np.random.permutation(len(t))
            split_index = int(len(t) * 0.8)
            train_indices = indices[:split_index]
            val_indices = indices[split_index:]

            tensors_train.append(t[train_indices])
            tensors_val.append(t[val_indices])
            
        dataset = {}
        dataset['train'] = np.vstack(tensors_train)
        dataset['val'] = np.vstack(tensors_val)

        print(dataset['train'].shape, dataset['val'].shape)

        save_model_path = os.path.join(save_path, model_name.replace('/', '--'), '_'.join(filters), f'd{d}')
        os.makedirs(save_model_path, exist_ok = True)

        mean_vector = dataset['train'].mean(axis=0)
        mean_value = dataset['train'].mean()
        std_vector = dataset['train'].std(axis=0)
        std_value = dataset['train'].std()
        # print(mean_value, std_value)
        
        filename = os.path.join(save_model_path, '_'.join(filters) + f'_d{d}_train_{chunk_direction}.tfrecord')
        np.save(filename.replace('.tfrecord', '_mean_vector.npy'), mean_vector)
        np.save(filename.replace('.tfrecord', '_std_vector.npy'), std_vector)
        np.save(filename.replace('.tfrecord', '_mean.npy'), mean_value)
        np.save(filename.replace('.tfrecord', '_std.npy'), std_value)

        for split in ['train', 'val']:
            print(f'## Saving {split} dataset ##')
            filename = os.path.join(save_model_path, '_'.join(filters) + f'_d{d}_{split}_{chunk_direction}.tfrecord')
            with tf.io.TFRecordWriter(filename) as writer:
                for slice in tqdm(dataset[split]):
                    example = serialize_example(slice)
                    writer.write(example)
                    

meta-llama/Meta-Llama-3-8B
dim = 4096


100%|██████████| 291/291 [00:00<00:00, 10659.67it/s]


model-layers-0-self_attn-q_proj-weight.npy
(3276, 4096) (820, 4096)
## Saving train dataset ##


100%|██████████| 3276/3276 [00:02<00:00, 1609.98it/s]


## Saving val dataset ##


...

100%|██████████| 291/291 [00:00<00:00, 2525.09it/s]


model-layers-0-self_attn-q_proj-weight.npy
(838860, 16) (209716, 16)
## Saving train dataset ##


100%|██████████| 838860/838860 [00:12<00:00, 65535.98it/s]


## Saving val dataset ##


100%|██████████| 209716/209716 [00:03<00:00, 65120.19it/s]


In [ ]:
n = np.load('/home/jgryu/Weight_compression/Wparam_dataset/TFRecord/meta-llama/Meta-Llama-3-8B/mlp_layers-0_down_d4096_train_mean.npy')

array(-2.1057717e-06, dtype=float32)

In [ ]:
## 불려오기
def parse_example(example_proto):
    feature_description = {
        'slice': tf.io.FixedLenFeature([d], tf.float32),  # feature 크기만큼 설정
    }
    return tf.io.parse_single_example(example_proto, feature_description)

dataset = tf.data.TFRecordDataset(filename)
dataset = dataset.map(parse_example)

for record in dataset:
    print(record['slice'])  # feature만 출력


# 텐서별로 tfrecord 만들기

In [3]:
model_names = ['meta-llama/Meta-Llama-3-8B']
tensor_zoo_path = '/home/jgryu/Weight_compression/Wparam_dataset/model_param_tensor/'
save_path = '/home/jgryu/Weight_compression/Wparam_dataset/TFRecord/'

# # model_name = model_names[0]
# model_name = model_names[0]

In [4]:
import glob, os
import random
import json
import tensorflow as tf
from tqdm import tqdm
import numpy as np

def check_contains_all_substrings(string, substrings):
    return all(substring in string for substring in substrings)

def check_contains_any_substrings(string, substrings):
    return any(substring in string for substring in substrings)

def serialize_example(slice):
    feature_dict = {
        'slice': tf.train.Feature(float_list=tf.train.FloatList(value=slice))
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature_dict))
    return example_proto.SerializeToString()

chunk_direction = 'out' ## 같은 Out channel 요소들이 같이 묶이게 차름
# chunk_direction = 'in'

for model_name in model_names:
    print(model_name)
    for d in [16]:
        print(f'dim = {d}')
        target_model_tensor_path = tensor_zoo_path + model_name
        tensor_path_list = glob.glob(f'{target_model_tensor_path}/**/*.npy', recursive=True)

        for tensor_path in tqdm(tensor_path_list):
            if 'embed' in tensor_path or 'layernorm' in tensor_path or 'head' in tensor_path: continue
            
            random.seed(100)
            tensors_train = []
            tensors_val = []
            
            save_model_path = os.path.join(
                save_path, 
                model_name.replace('/', '--'), 
                'per_tensor', 
                tensor_path.split('/')[-1].replace('.npy', ''), 
                f'd{d}'
            )
            if os.path.exists(save_model_path):
                continue
            os.makedirs(save_model_path, exist_ok=True)
            
            print(tensor_path.split('/')[-1])
            t = np.load(tensor_path)
            # print(t.shape)
            if chunk_direction == 'in':
                t = t.T
            t = t.astype(np.float32)
            t = t.reshape(-1, d)
            # print(t.shape)
            # all_tensors.append(t)
            
            indices = np.random.permutation(len(t))
            split_index = int(len(t) * 0.8)
            train_indices = indices[:split_index]
            val_indices = indices[split_index:]

            tensors_train.append(t[train_indices])
            tensors_val.append(t[val_indices])
                
            dataset = {}
            dataset['train'] = np.vstack(tensors_train)
            dataset['val'] = np.vstack(tensors_val)

            print(dataset['train'].shape, dataset['val'].shape)

            mean_vector = dataset['train'].mean(axis=0)
            mean_value = dataset['train'].mean()
            std_vector = dataset['train'].std(axis=0)
            std_value = dataset['train'].std()
            # print(mean_value, std_value)
            
            filename = os.path.join(save_model_path, tensor_path.split('/')[-1].replace('.npy', '') + f'_d{d}_train_{chunk_direction}.tfrecord')
            np.save(filename.replace('.tfrecord', '_mean_vector.npy'), mean_vector)
            np.save(filename.replace('.tfrecord', '_std_vector.npy'), std_vector)
            np.save(filename.replace('.tfrecord', '_mean.npy'), mean_value)
            np.save(filename.replace('.tfrecord', '_std.npy'), std_value)

            for split in ['train', 'val']:
                print(f'## Saving {split} dataset ##')
                filename = os.path.join(save_model_path, tensor_path.split('/')[-1].replace('.npy', '') + f'_d{d}_{split}_{chunk_direction}.tfrecord')
                with tf.io.TFRecordWriter(filename) as writer:
                    for slice in tqdm(dataset[split]):
                        example = serialize_example(slice)
                        writer.write(example)
                        

meta-llama/Meta-Llama-3-8B
dim = 16


  0%|          | 0/291 [00:00<?, ?it/s]

model-layers-28-self_attn-k_proj-weight.npy
(209715, 16) (52429, 16)
## Saving train dataset ##


100%|██████████| 209715/209715 [00:02<00:00, 71409.53it/s]


## Saving val dataset ##


...

100%|█████████▉| 290/291 [1:42:16<00:43, 43.49s/it]

model-layers-7-self_attn-q_proj-weight.npy
(838860, 16) (209716, 16)
## Saving train dataset ##


100%|██████████| 838860/838860 [00:11<00:00, 71866.83it/s]


## Saving val dataset ##


100%|██████████| 291/291 [1:42:31<00:00, 21.14s/it]
